In [22]:
import pandas as pd
import sqlite3
import sqlalchemy 
import json
from datasets import load_dataset

In [ ]:
DB_PATH = "./subset_selection/sst_results.db"
with sqlite3.connect(DB_PATH) as conn: 
    #Now in order to read in pandas dataframe we need to know table name
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    table_name = cursor.fetchall()[0][0]
    print(f"Table Name : {table_name}")

    df = pd.read_sql_query(f'SELECT * FROM {table_name}', conn)

In [4]:
df

,indexes,objective
0,"[787, 882, 40, 341, 477, 454, 227, 334, 837, 2...",0.000000
1,"[787, 882, 40, 341, 477, 454, 227, 334, 837, 2...",0.690498
2,"[787, 882, 40, 341, 477, 454, 227, 334, 837, 2...",0.697738
3,"[787, 882, 40, 341, 477, 454, 227, 334, 837, 2...",0.690950
4,"[787, 882, 40, 341, 477, 454, 227, 334, 837, 2...",0.709502
...,...,...
8566,"[517, 882, 82, 341, 477, 381, 611, 377, 837, 2...",0.747964
8567,"[48, 882, 82, 341, 477, 381, 611, 377, 837, 21...",0.752941
8568,"[517, 882, 82, 341, 477, 381, 611, 377, 837, 2...",0.742081
8569,"[517, 882, 82, 341, 477, 381, 611, 377, 837, 2...",0.751131


In [5]:
df['objective'].idxmax()

7227

In [34]:
optimal_subset = set(json.loads(df.iloc[7227].indexes))

In [36]:
len(optimal_subset)

96

In [11]:
indexes_list = list(map(json.loads, df["indexes"].to_list()))
indexes_list = [item for sublist in indexes_list for item in sublist]
indexes_set = set(indexes_list)

In [25]:
sst2 = load_dataset("sst")

No config specified, defaulting to: sst/default
Reusing dataset sst (/Users/garylai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)
100%|██████████| 3/3 [00:00<00:00, 405.23it/s]


In [27]:
data_pool = sst2["train"].shuffle(seed=0).select(range(1000))

In [30]:
len(data_pool["sentence"])

1000

In [ ]:
def preprocess(data):
    data = data.rename_column("label", "scalar_label")
    data = data.map(lambda x: {"label": 1 if data in optimal_subset else 0})
    


In [ ]:
# preprocess the data in data pool